In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [23]:
dec_2016 = pd.read_csv('data/raw_data/Video_Games_Sales_as_at_22_Dec_2016.csv')
jan_2017 = pd.read_csv('data/raw_data/Video_Game_Sales_as_of_Jan_2017.csv')

In [24]:
''' Read all the csv's into a list. '''
year_csv = []
for csv in glob.iglob('data/time_series_data/*.csv'):
    year_csv.append(csv)